In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
from tqdm import tqdm
import os


from holodeck import plot, detstats, utils
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

In [ ]:
SHAPE = None
NREALS = 500
NFREQS = 40
NLOUDEST = 10
BGL = 1

TOL=0.01
MAXBADS=5

NVARS = 21

NPSRS = 40
NSKIES = 100

ALL_TARGETS = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

# Plot Number of Sources Contributing to EV_SS

In [ ]:
BUILD_ARRAYS=False

ylabel='$N(\mathrm{DP_{SS}}>x)$'
title = 'Number of Sources Contributing to $\langle N \\rangle_\mathrm{SS}$'

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', yscale='linear', height=7)
fig.text(0.08, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, y1, y2, _, _ = detstats.build_noise_arrays(target)
    else:
        _, y1, y2, _, _ = detstats.get_noise_arrays_temp(target)

    y1 = y1.reshape(NVARS, NSKIES*NREALS)
    y2 = y2.reshape(NVARS, NREALS*NSKIES)

    h1 = plot.draw_med_conf_color(ax, xx, y1, color='tab:orange')
    h2 = plot.draw_med_conf_color(ax, xx, y2, color='tab:blue')

plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['$0.50$', '$0.01$']
fig.legend(handles=[h1, h2], labels=labels, bbox_to_anchor=(0.5,-0.02), loc='lower center',
           ncols=2, title='$x$ threshold')

# Plot Noise

In [ ]:
target='gsmf_phi0'
white_noise, count50, count01, hc_ss, hc_bg = detstats.get_noise_arrays_temp(target)

fobs, _ = holo.utils.pta_freqs()
psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                          reshape_freqs=False) # V,F,R,L

# white_noise, count_cws = detstats.build_noise_arrays(target)

In [ ]:
BUILD_ARRAYS=False

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        white_noise, _, _, hc_ss, hc_bg = detstats.build_noise_arrays(target)
    else:
        white_noise, _, _, hc_ss, hc_bg= detstats.get_noise_arrays_temp(target)

    print(white_noise.shape)
    psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
    psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                          reshape_freqs=False) # V,F,R,L

    yy_wn = white_noise.reshape(NVARS, NREALS)
    yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
    yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

    yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
    yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
    yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

    h1 = plot.draw_med_conf_color(ax, xx, yy_wn, color='grey', linestyle=':')
    h2 = plot.draw_med_conf_color(ax, xx, yy_bg, color='tab:blue', linestyle='--')
    h3 = plot.draw_med_conf_color(ax, xx, yy_ss, color='black', fracs=[0.5,])
    h4 = plot.draw_med_conf_color(ax, xx, yy_l1, color='#00ff00')
    h5 = plot.draw_med_conf_color(ax, xx, yy_l2, color='#ff0000' )
    h6 = plot.draw_med_conf_color(ax, xx, yy_lt, color='#4000ff')



plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
fig.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=6, columnspacing=1.5)